In [4]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import re
from time import sleep
import acquire_r as ac

In [75]:
url = 'https://www.pro-football-reference.com/boxscores/202011120oti.htm'
response = requests.get(url)
response

<Response [200]>

In [76]:
#1
soup = BeautifulSoup(response.content, 'html.parser')
#2
div_grid = soup.find_all('div', class_='content_grid')[0]
#3
div = div_grid.find_all('div')[0]
#4
div_wrap = div.find_all('div')[0]
#5
soup_wrap4 = div_wrap.contents[4]
#6
soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
#7
wx = soup4.find_all('td','center')[3]

In [77]:
wx.contents[0]

'grass'

## Attempt to start scraping

In [114]:
## scale weekly
url = 'https://www.pro-football-reference.com/years/2022/week_18.htm'



In [115]:
response = requests.get(url)
response

<Response [200]>

In [116]:
soup = BeautifulSoup(response.content, 'html.parser')

In [117]:
html_links = soup.find_all('td', class_='right gamelink')

In [118]:
html_links[0].find('a')['href']

'/boxscores/202301070rai.htm'

In [119]:
## list of games
games = [item.find('a')['href'] for item in html_links]
games

['/boxscores/202301070rai.htm',
 '/boxscores/202301070jax.htm',
 '/boxscores/202301080atl.htm',
 '/boxscores/202301080buf.htm',
 '/boxscores/202301080chi.htm',
 '/boxscores/202301080cin.htm',
 '/boxscores/202301080clt.htm',
 '/boxscores/202301080mia.htm',
 '/boxscores/202301080nor.htm',
 '/boxscores/202301080pit.htm',
 '/boxscores/202301080den.htm',
 '/boxscores/202301080phi.htm',
 '/boxscores/202301080sea.htm',
 '/boxscores/202301080sfo.htm',
 '/boxscores/202301080was.htm',
 '/boxscores/202301080gnb.htm']

In [120]:
## list of weeks
weeks = [18 for i in games]
weeks

[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]

In [124]:
turf = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[3]
    turfs = str(wx.contents[0])
    turf.append(turfs)

In [125]:
turf

['grass',
 'grass ',
 'fieldturf',
 'a_turf',
 'grass ',
 'fieldturf ',
 'fieldturf ',
 'grass ',
 'sportturf',
 'grass ',
 'grass ',
 'grass ',
 'outdoors',
 'grass',
 'grass ',
 'grass ']

## list of seasons

In [127]:
date_re = r'[0-9]{4}'
re.findall(date_re,games[0])[0]
season = re.findall(date_re,games[0])[0]
seasons = [re.findall(date_re,i)[0] for i in games]
seasons

['2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023']

## list of homes

In [128]:
home_re = r'[a-z]{3}'
home = re.findall(home_re,games[0])[3]
homes = [re.findall(home_re,i)[3] for i in games]
homes

['rai',
 'jax',
 'atl',
 'buf',
 'chi',
 'cin',
 'clt',
 'mia',
 'nor',
 'pit',
 'den',
 'phi',
 'sea',
 'sfo',
 'was',
 'gnb']

In [137]:
home_teams = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    href = soup.find_all('div', class_='scorebox')[0].find_all('a')[8]
    team = href.text
    home_teams.append(team)
home_teams

['Las Vegas Raiders',
 'Prev Game',
 'Atlanta Falcons',
 'Prev Game',
 'Chicago Bears',
 'Cincinnati Bengals',
 'Prev Game',
 'Prev Game',
 'Prev Game',
 'Prev Game',
 'Denver Broncos',
 'Philadelphia Eagles',
 'Prev Game',
 'Prev Game',
 'Washington Commanders',
 'Prev Game']

## Dataframe

In [139]:
df = pd.DataFrame({
    'schedule_season':seasons,
    'schedule_week':weeks,
    'team_home':home_teams,
    'turf_type':turf,
})

In [140]:
df

,schedule_season,schedule_week,team_home,turf_type
0,2023,18,Las Vegas Raiders,grass
1,2023,18,Prev Game,grass
2,2023,18,Atlanta Falcons,fieldturf
3,2023,18,Prev Game,a_turf
4,2023,18,Chicago Bears,grass
5,2023,18,Cincinnati Bengals,fieldturf
6,2023,18,Prev Game,fieldturf
7,2023,18,Prev Game,grass
8,2023,18,Prev Game,sportturf
9,2023,18,Prev Game,grass


In [141]:
table = soup.find_all('div', class_='content_grid')[0].find_all('div')[0].find_all('div')[0].contents[4]

In [142]:
soup1 = BeautifulSoup(table, 'html.parser')

In [143]:
soup1.find_all('td')[3].contents[0]

'grass '